**Modelo de Prediccion de accidentes automovilísticos en autopista**

*Caso de estudio*: Predecir si un accidente automovilístico se provocó en una autopista o no. Para la predicción de los accidentes, es importante tener en cuenta las siguientes variables:

* Dia de la Semana: Día en que ocurre el evento
* ViolCat: Nivel de impacto del accidente
* Mes: mes en que ocurre el evento
* Cantidad de Luz Diurna: si en el momento que ocurrio el accidente habia luz del dia o no
* Cielo Despejado:Indica si el cielo estaba despejado o no


La variable de referencia sera la variable Highway. Esta variable indica si el accidente fue en una autopista o no.

1. Se cargan las librerias de trabajo.

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

2.se cargan los datos de trabajo

In [3]:
nxl= '/BD2_CarCrash.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0,engine='openpyxl')#control,shify enter #lee la base de dato

TD=len(XDB)#TOTAL DE LOS DATOS
print("\n la cantidad de datos totales es:",TD)

XDB=XDB[XDB['County']=='SAN FRANCISCO'] #filtro el country
XDB=XDB[XDB['City']=='SAN FRANCISCO']#filtro el city
XDB['CrashType_N']=XDB['CrashType'].astype('category').cat.codes #convercion de la variable CrashType a un valor numerico y se renombre como CrashType_num
TD_SF=len(XDB)#total de datos de san francisco
print("\n la cantidad de datos totales de San Francisco es:",TD_SF)

#seleccionamos las variables de trabajo -- minimiza la base datos
XDB=XDB[['Weekday','ViolCat','ClearWeather','Month','CrashType_N', 'Daylight', 'Highway']]#aca coloco country y city para verificar que si me filtro y despues los retiro


filas=np.where(~np.isnan(XDB['Highway']))[0]# busca los valores nulos en la columna Highway
XDB=XDB.iloc[filas,:]# elimina los valores nulos (true) de la columna Highway
XDB=XDB.dropna() #evita datos faltantes del resto de las columnas


#separa variables de entrada - variables de salida
YDB = XDB['Highway'] #SALIDA
XDB = XDB[['Weekday','ViolCat','ClearWeather','Month','CrashType_N', 'Daylight']] #ENTRADA


 la cantidad de datos totales es: 112660

 la cantidad de datos totales de San Francisco es: 2081


3. se implementan el modelo de clasificacion Naive Bayes

In [4]:
mnb=GaussianNB()#los dato siguen una distribucion gaussiana - es formala para poder trabajar con NB
mnb.fit(XDB,YDB) #entrena el modelo para que trabaje con los datos de la base de datos - es formula para poder trabajr con NB

# datos por categoria
ndat= mnb.class_count_ # ndat es una variables que me va hacer el conteo de datos por categoria [0, 1]
print("el numero de datos por categoria es: \n" , ndat)
print("accidentes en otras vias: " , ndat[0])
print("accidentes en autopistas: " , ndat[1])
print("porcentaje otras vias: " , ndat[0]/len(XDB))
print("porcentaje autopistas: " , ndat[1]/len(XDB))

cat=mnb.classes_
print("\n las categorias son:\n" , cat)#muestra las categorias con las que estamos trabajo[0,1]
# determinamos la propiedades estadisticas de las variables
u=mnb.theta_
print("\n La media de las variables es: \n",u) #con esto calculamos media

sigma=np.sqrt(mnb.var_)#con esto calculo la desviacion estadar --- la raiz de la varainza
print("\n La desviacion estandar de las variables es: \n",sigma)
print("\n Los limites superiores las variables es: \n", u+sigma) #desviacion+media=limite inferior
print("\n Los limites inferiores de las variables es: \n", u-sigma)#desviacion-media=limite inferior

el numero de datos por categoria es: 
 [1580.  501.]
accidentes en otras vias:  1580.0
accidentes en autopistas:  501.0
porcentaje otras vias:  0.7592503604036521
porcentaje autopistas:  0.2407496395963479

 las categorias son:
 [0 1]

 La media de las variables es: 
 [[3.88417722 7.45696203 0.86265823 4.35189873 3.21772152 0.66265823]
 [3.9261477  4.1497006  0.79840319 6.69461078 2.27744511 0.62075848]]

 La desviacion estandar de las variables es: 
 [[1.95062542 3.52136997 0.34420782 2.3433645  1.87371962 0.47280262]
 [2.01603723 2.16198902 0.40119266 3.44163641 1.05765448 0.48519831]]

 Los limites superiores las variables es: 
 [[ 5.83480263 10.978332    1.20686605  6.69526323  5.09144114  1.13546084]
 [ 5.94218494  6.31168962  1.19959585 10.13624719  3.33509959  1.10595679]]

 Los limites inferiores de las variables es: 
 [[1.9335518  3.93559205 0.51845041 2.00853424 1.3440019  0.18985561]
 [1.91011047 1.98771158 0.39721054 3.25297437 1.21979063 0.13556017]]


4. se procede con la evaluacion de desempeño del modelo(frente al pronostico de la prediccion)

In [5]:
ydp=mnb.predict(XDB)# es formula y el XDB es la entrada
cm=confusion_matrix(YDB,ydp) #YDB me pone los 1,0 de la matriz osea las salida, y ydp son los valores que se clasifican dentro de la matriz
print("La matriz de confusion es: \n",cm)

La matriz de confusion es: 
 [[1365  215]
 [ 224  277]]


5. Metricas de desempeño

In [6]:
cm=confusion_matrix(YDB,ydp)#esta es la matiz de confunsion
print("La matriz de confusion es: \n",cm) # imprime
VN=cm[0,0];FP=cm[0,1] #VN sera los valores de cm(la matriz) que sean igual a (0,0)
FN=cm[1,0];VP=cm[1,1]
TD=len(XDB)#TOTAL DE LOS DATOS
print("\n la cantidad de datos es:",TD)

#EXACTITUD: MUESTRA EL COMPORTAMIENTO GENERAL DEL MODELO
Ex=(VN+VP)/TD; print("\n la exactitud es:",Ex)

#tasa de error: porcentaje de equivocacion
Ter=(FN+FP)/TD; print("\n la tasa de error es:",Ter)

#sensibilidad: como se comporta el modelo frente a los accientes en autopista
Sen=VP/(VP+FN); print("\n la sensibilidad es:",Sen)#aca mira la probabilida de que sea preapobado CON VP y FP

#Especificidad: como se comporta el modelo frente a los accidentes en otras vias
Esp=VN/(VN+FP); print("\n la especificidad es:",Esp) #aca mira la probabilida de que sea preapobado CON VN y FN

#precision: capacidad del modelo para predecir VP
Pr=VP/(VP+FP); print("\n la precision es:",Pr)#aca mira la probabilida de que sea VP(accidentes en vias)

#prediccion negativa: capacidad del modelo para predecir vn
prn=VN/(VN+FN); print("\n la prediccion negativa es:",prn)#aca mira la probabilida de que sea VN(accidentes en otras vias)

La matriz de confusion es: 
 [[1365  215]
 [ 224  277]]

 la cantidad de datos es: 2081

 la exactitud es: 0.7890437289764536

 la tasa de error es: 0.21095627102354636

 la sensibilidad es: 0.5528942115768463

 la especificidad es: 0.8639240506329114

 la precision es: 0.5630081300813008

 la prediccion negativa es: 0.8590308370044053


6.evaluamos el procentaje de tener un accidente en la Autopista o No segun las caracteristicas de una persona

In [7]:
XI=[5,1,1,3,4,0]
ydpi=mnb.predict([XI])
ydpip=mnb.predict_proba([XI])
#print("La probabilidad de que esta persona tenga o no un accidente en la autopista:",ydpip)
print("La persona tuvo un accidente en la autopista:",ydpi)

print(f"\n Probabilidad NO autopista: {ydpip[0][0]*100:.2f}%")
print(f"\n Probabilidad SÍ autopista: {ydpip[0][1]*100:.2f}% \n")

La persona tuvo un accidente en la autopista: [0]

 Probabilidad NO autopista: 82.86%

 Probabilidad SÍ autopista: 17.14% 



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


**Analisis de Resultados**

* De acuerdo con la base de datos se puede identificar un total de 112600 datos, los cuales 2081 pertenecen la ciudad de San Francisco.
*Adiccionalmente podemos observar que de los 2081 datos que pertenecen a la ciudad de san francisco, 501 fueron accidentes en autopista y 1580 fueron en otras vias. lo que nos permite determinar que fueron mas los accientes en otras vias en la ciudad de san francisco.

* De la matriz de confusion se puede observar que el modelo tiene la tendecnia a equivocarse en mayor medida en la prediccion de los valores de accidentes en autopistas con un total de 224 falsos negativos, frente a un 215 de falsos positivos.

* la probabilidad de que una persona de san francisco tenga un accidente en autopistas del 24%

* la probabilidad de que una persona de san francisco tenga un accidente en otras vias es del 75,9%

* de acuerdo con las metricas que arroja la matriz de confusion, se puede observar que el modelo alcanzo un exactitud del 78% lo cual esta por arriba de 75%, lo que muestra el buen comportamiento del modelo frente a la predecion de accidentes en autopistas.

* La tasa de error del modelo corresponde a un 21%

* El modelo tiende a predecir mas accidentes en otras vias que en aoutopistas, ya que tenemos un sensibilidad del 55% y una especifidad el 86%

* se destaca tambien el buen funcionamiento alcanzado por el modelo en el pronostivo de los verdaderos negativos (prediccion negativa)el cual alcanzo un porcentaje de 85% . este porcentaje fue mas alto frente al pronostico de los verdaderos positivos, donde se alcanzo una precision de 56% en promedio

* finalmente pudimos evidenciar que  la probabilidad de que una persona con las siguientes características ['Weekday','ViolCat','ClearWeather','Month','CrashType_N','Daylight']=[5,1,1,3,4,0] , tenga o no tenga una accidente en autopista  es de 17.14% a que tenga un accidente en una autopista y un 82.86% de que no tenga un accidente en una autopista

In [8]:
nxl= '/BD2_CarCrash.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0,engine='openpyxl')#control,shify enter #lee la base de dato

XDB=XDB[XDB['County']=='SAN FRANCISCO'] #filtro el country
XDB=XDB[XDB['City']=='SAN FRANCISCO']#filtro el city
XDB['CrashType_N']=XDB['CrashType'].astype('category').cat.codes #convercion de la variable CrashType a un valor numerico y se renombre como CrashType_num
TD_SF=len(XDB)#total de datos de san francisco
print("\n la cantidad de datos totales de San Francisco es:",TD_SF)

#seleccionamos las variables de trabajo -- minimiza la base datos
XDB=XDB[['Weekday','ViolCat','ClearWeather','Month','CrashType_N', 'Daylight', 'Highway']]#aca coloco country y city para verificar que si me filtro y despues los retiro


filas_sp=np.where(~np.isnan(XDB['Highway']))[0]# busca los valores nulos en la columna Highway
XDBsp=XDB.iloc[filas_sp,:]# elimina los valores nulos (true) de la columna Highway
XDBsp=XDBsp[['Weekday','ViolCat','ClearWeather','Month','CrashType_N', 'Daylight']] #evita datos faltantes del resto de las columna

#procedemos con el pronostico
ydpsp = mnb.predict(XDBsp)

#pegamos el pronostico
TAsq = np.column_stack((filas_sp,XDBsp , ydpsp))
df = pd.DataFrame(TAsq)
df.columns = ['PersonaEvaluada','Weekday','ViolCat','ClearWeather','Month','CrashType_N', 'Daylight', 'Highway']
display(df)
df.to_excel("ResultadosPredicionAccidentes.xlsx")


 la cantidad de datos totales de San Francisco es: 2081


,PersonaEvaluada,Weekday,ViolCat,ClearWeather,Month,CrashType_N,Daylight,Highway
0,0,7,1,1,3,0,1,0
1,1,2,9,1,2,0,0,0
2,2,7,3,1,1,0,0,0
3,3,7,3,1,3,0,0,0
4,4,3,3,1,3,0,0,0
...,...,...,...,...,...,...,...,...
2076,2076,5,11,1,8,6,0,0
2077,2077,5,11,1,3,6,1,0
2078,2078,6,10,0,4,6,0,0
2079,2079,4,10,1,4,6,0,0
